## 1. Setup Iniziale

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## 2. MCP

### Server locali

In [ ]:
# import sys
# import asyncio

# # Fix for Windows issues in Jupyter notebooks
# if sys.platform == "win32":
#     # 1. Use ProactorEventLoop for subprocess support
#     if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
#         asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
#     # 2. Redirect stderr to avoid fileno() error when launching MCP servers
#     if "ipykernel" in sys.modules:
#         sys.stderr = sys.__stderr__


In [ ]:
# from langchain_mcp_adapters.client import MultiServerMCPClient

# client = MultiServerMCPClient(
#     {
#         "local_server": {
#                 "transport": "stdio",
#                 "command": "python",
#                 "args": ["../data/resources/2.1_mcp_server.py"],
#             }
#     }
# )

In [ ]:
# # get tools
# tools = await client.get_tools()

# # get resources
# resources = await client.get_resources("local_server")

# # get prompts
# prompt = await client.get_prompt("local_server", "prompt")
# prompt = prompt[0].content

In [ ]:
# from langchain.agents import create_agent
# from langchain_cerebras import ChatCerebras

# model=ChatCerebras(model="gpt-oss-120b")

# agent = create_agent(
#     model=model,
#     tools=tools,
#     system_prompt=prompt
#)

In [ ]:
# from langchain.messages import HumanMessage

# config = {"configurable": {"thread_id": "1"}}

# response = await agent.ainvoke(
#     {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
#     config=config
# )

In [ ]:
# for r in response['messages']:
#     r.pretty_print()

### Server Online

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
    "kiwi-com-flight-search": {
        "transport": "http",
        "url": "https://mcp.kiwi.com"
    }
    }
)
tools = await client.get_tools()

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_cerebras import ChatCerebras

model=ChatCerebras(model="gpt-oss-120b")

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="You are a travel assistant. Help users find flights based on their requests. No follow up questions. Answer in the user's language. Maximum 200 words. Keep the response of the tool under 500 words"
)

In [ ]:
from langchain.messages import HumanMessage
config = {"configurable": {"thread_id": "flight_search_1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Trovami un volo da New York a Parigi il 25 marzo 2026")]},
    config=config
)

In [ ]:
for r in response['messages']:
    r.pretty_print()

## 3. Wrapper dinamici

### Modelli dinamici

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable
from langchain_cerebras import ChatCerebras

large_model = ChatCerebras(model="gpt-oss-120b")
standard_model = ChatCerebras(model="llama-3.3-70b")

@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on State conversation length."""
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    else:
        # Short conversation - use efficient model
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=standard_model,
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)

print(response["messages"][-1].content)

In [ ]:
print(response["messages"][-1].response_metadata["model_name"])

In [ ]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

In [ ]:
print(response["messages"][-1].response_metadata["model_name"])

### Prompt dinamici

In [ ]:
from dataclasses import dataclass
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dataclass
class LanguageContext:
    user_language: str = "English"

@dynamic_prompt
def user_language_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_language = request.runtime.context.user_language
    base_prompt = "You are a helpful assistant."

    if user_language != "English":
        return f"{base_prompt} only respond in {user_language}."
    elif user_language == "English":
        return base_prompt

In [ ]:
from langchain.agents import create_agent
from langchain_cerebras import ChatCerebras

model = ChatCerebras(model="gpt-oss-120b")

agent = create_agent(
    model=model,
    context_schema=LanguageContext,
    middleware=[user_language_prompt]
)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Irish")
)

print(response["messages"][-1].content)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Spanish")
)

print(response["messages"][-1].content)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="French")
)

print(response["messages"][-1].content)

### Tools dinamici

In [ ]:
from langchain.tools import tool
from typing import Dict, Any
from langchain_community.utilities import SQLDatabase
from langchain_tavily import TavilySearch  # updated at 1.0

db = SQLDatabase.from_uri("sqlite:///../data/resources/Chinook.db")

tavily_search = TavilySearch(max_results=3)

data = tavily_search.invoke({"query": "What is LangGraph?"})
search_docs = data.get("results", data)

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_search.invoke({"query": query})

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [ ]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [ ]:
from langchain.agents import create_agent
from langchain_cerebras import ChatCerebras

model = ChatCerebras(model="gpt-oss-120b")

agent = create_agent(
    model=model,
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)